# Quantum Error Correction with Loom & Deltakit

## A Comprehensive Tutorial

This notebook demonstrates practical quantum error correction (QEC) using two leading frameworks:
- **Loom** by Entropica Labs - Focus on design and lattice surgery
- **Deltakit** by Riverlane - Focus on learning and deployment

### Prerequisites
```bash
pip install loom deltakit numpy matplotlib
```

For Deltakit, you'll also need a free token from https://deltakit.riverlane.com/

---

## Table of Contents

### Part 1: Introduction to QEC
1. [Why Quantum Error Correction?](#why-qec)
2. [Basic QEC Concepts](#qec-concepts)

### Part 2: Loom Examples
3. [Loom: Repetition Code](#loom-repetition)
4. [Loom: Surface Code](#loom-surface)
5. [Loom: Lattice Surgery](#loom-lattice)
6. [Loom: Custom Circuits with EKA](#loom-eka)

### Part 3: Deltakit Examples
7. [Deltakit: Setup & Repetition Code](#deltakit-setup)
8. [Deltakit: Memory Experiment](#deltakit-memory)
9. [Deltakit: Decoder Comparison](#deltakit-decoders)
10. [Deltakit: qLDPC Codes](#deltakit-qldpc)

### Part 4: Comparative Analysis
11. [Side-by-Side Comparison](#comparison)
12. [Threshold Analysis](#threshold)
13. [Performance Benchmarks](#benchmarks)

---

<a id='why-qec'></a>
## 1. Why Quantum Error Correction?

Quantum computers are extremely sensitive to noise. Unlike classical bits that are either 0 or 1, qubits exist in superposition and are affected by:
- **Decoherence**: Loss of quantum information over time
- **Gate errors**: Imperfect quantum operations
- **Measurement errors**: Incorrect readout of qubit states

### The Challenge
Without error correction:
- Typical gate fidelity: 99.9% (0.1% error rate)
- After 1,000 gates: Only ~36% chance of success
- After 10,000 gates: Essentially 0% success rate

### The Solution: QEC
Quantum error correction encodes logical qubits into multiple physical qubits, allowing:
1. Detection of errors without destroying quantum information
2. Correction of errors in real-time
3. Logical error rates much lower than physical error rates

**Key Result**: Below a threshold error rate (~1% for surface codes), increasing code distance exponentially reduces logical errors!

In [ ]:
# Visualization: Error accumulation without QEC
import numpy as np
import matplotlib.pyplot as plt

num_gates = np.arange(0, 10001, 100)
physical_error_rate = 0.001  # 0.1% per gate

# Without QEC: errors accumulate
success_rate_no_qec = (1 - physical_error_rate) ** num_gates

# With QEC: logical error rate is suppressed
logical_error_rate_d3 = physical_error_rate ** 2  # Distance-3 scaling
logical_error_rate_d5 = physical_error_rate ** 3  # Distance-5 scaling
success_rate_qec_d3 = (1 - logical_error_rate_d3) ** num_gates
success_rate_qec_d5 = (1 - logical_error_rate_d5) ** num_gates

plt.figure(figsize=(12, 6))
plt.semilogy(num_gates, success_rate_no_qec, label='No QEC', linewidth=2)
plt.semilogy(num_gates, success_rate_qec_d3, label='QEC Distance-3', linewidth=2)
plt.semilogy(num_gates, success_rate_qec_d5, label='QEC Distance-5', linewidth=2)
plt.xlabel('Number of Gates', fontsize=12)
plt.ylabel('Success Probability', fontsize=12)
plt.title('Quantum Error Correction Impact', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"After 10,000 gates:")
print(f"  No QEC: {success_rate_no_qec[-1]:.6e} success rate")
print(f"  Distance-3 QEC: {success_rate_qec_d3[-1]:.6f} success rate")
print(f"  Distance-5 QEC: {success_rate_qec_d5[-1]:.6f} success rate")

<a id='qec-concepts'></a>
## 2. Basic QEC Concepts

### Key Terminology

- **Physical Qubit**: The actual noisy qubit in hardware
- **Logical Qubit**: Error-protected qubit encoded in multiple physical qubits
- **Code Distance (d)**: Minimum number of errors needed to cause a logical error
- **Stabilizers**: Operators that reveal error information without collapsing the quantum state
- **Syndrome**: The measurement outcome from stabilizers indicating what errors occurred
- **Decoder**: Algorithm that determines corrections from syndrome measurements

### Common QEC Codes

| Code | Physical Qubits | Distance | Best For |
|------|----------------|----------|----------|
| Repetition | d | d | Simple bit-flip protection |
| Shor | 9 | 3 | Historical importance |
| Steane | 7 | 3 | Fault-tolerant gates |
| Surface | d² + (d-1)² | d | Near-term hardware |
| qLDPC | O(d) | d | Long-term scaling |

### The QEC Workflow

```
1. Encode: Logical |0⟩ → Multiple physical qubits
2. Compute: Perform logical operations
3. Measure Syndromes: Detect errors
4. Decode: Determine likely errors
5. Correct: Apply corrections
6. Repeat: Steps 2-5 continuously
7. Decode: Final logical measurement
```

---
# Part 2: Loom Examples

Loom by Entropica Labs provides a flexible framework for designing and simulating QEC codes.

**Key Features:**
- EKA data structure (single source of truth)
- Entwine visual GUI for lattice surgery
- Multiple simulation backends
- Extensive lattice surgery support

---

<a id='loom-repetition'></a>
## 3. Loom: Repetition Code

The repetition code is the simplest QEC code. It encodes one logical qubit into multiple physical qubits to protect against bit-flip errors.

In [ ]:
# Note: This code is illustrative. Actual implementation may vary based on Loom version.

try:
    from loom.code_factory import RepetitionCodeFactory
    from loom.visualizer import plot_stabilizers
    
    print("Creating a distance-5 repetition code...")
    
    # Create the code
    code_factory = RepetitionCodeFactory(distance=5, orientation='horizontal')
    code = code_factory.generate_code()
    
    # Convert to EKA (Entropica's data structure)
    eka = code.to_eka()
    
    print(f"\nCode Properties:")
    print(f"  Distance: {code.distance}")
    print(f"  Data qubits: {len(eka.data_qubits)}")
    print(f"  Ancilla qubits: {len(eka.ancilla_qubits)}")
    print(f"  Stabilizers: {len(eka.stabilizers)}")
    
    # Visualize the stabilizer structure
    print("\nStabilizer operators:")
    for i, stab in enumerate(eka.stabilizers):
        print(f"  S{i}: {stab}")
    
    # Plot (if visualization available)
    try:
        plot_stabilizers(eka)
        plt.title('Repetition Code Stabilizers')
        plt.show()
    except:
        print("\nVisualization not available in this environment")
    
except ImportError:
    print("Loom not installed. Install with: pip install loom")
    print("\nRepetition Code Concept:")
    print("  Logical |0⟩ → |00000⟩")
    print("  Logical |1⟩ → |11111⟩")
    print("  Stabilizers: Z₀Z₁, Z₁Z₂, Z₂Z₃, Z₃Z₄")
    print("  Can correct up to 2 bit-flip errors")

### Building a Circuit with the Repetition Code

In [ ]:
try:
    from loom.circuit import Circuit
    from loom.interpretation import Interpretation
    
    # Create circuit
    circuit = Circuit(eka)
    
    # Initialize in logical |0⟩
    circuit.add_logical_initialization('Z')
    
    # Perform 3 QEC cycles
    num_cycles = 3
    for cycle in range(num_cycles):
        circuit.add_syndrome_measurement_round()
    
    # Final measurement
    circuit.add_logical_measurement('Z')
    
    # Create interpretation (maps to physical operations)
    interpretation = Interpretation(circuit)
    
    print(f"Circuit created:")
    print(f"  QEC cycles: {num_cycles}")
    print(f"  Circuit depth: {interpretation.depth}")
    print(f"  Total measurements: {interpretation.num_measurements}")
    print(f"  Total gates: {interpretation.num_gates}")
    
except (ImportError, NameError):
    print("Circuit building requires Loom installation")
    print("\nCircuit structure (conceptual):")
    print("  1. Initialize: |00000⟩")
    print("  2. Syndrome Round 1: Measure Z₀Z₁, Z₁Z₂, Z₂Z₃, Z₃Z₄")
    print("  3. Syndrome Round 2: Measure stabilizers again")
    print("  4. Syndrome Round 3: Measure stabilizers again")
    print("  5. Final: Measure all data qubits")

<a id='loom-surface'></a>
## 4. Loom: Surface Code

The surface code is the leading QEC code for near-term quantum computers. It has a high error threshold (~1%) and can be implemented on 2D qubit arrays.

In [ ]:
try:
    from loom.code_factory import RotatedSurfaceCodeFactory
    
    print("Creating a distance-3 rotated surface code...\n")
    
    # Create surface code
    surface_factory = RotatedSurfaceCodeFactory(distance=3)
    surface_code = surface_factory.generate_code()
    surface_eka = surface_code.to_eka()
    
    print(f"Surface Code Properties:")
    print(f"  Distance: {surface_code.distance}")
    print(f"  Total qubits: {len(surface_eka.data_qubits) + len(surface_eka.ancilla_qubits)}")
    print(f"  Data qubits: {len(surface_eka.data_qubits)}")
    print(f"  Ancilla qubits: {len(surface_eka.ancilla_qubits)}")
    print(f"  Total stabilizers: {len(surface_eka.stabilizers)}")
    
    # Count X and Z stabilizers
    x_stabilizers = sum(1 for s in surface_eka.stabilizers if 'X' in str(s))
    z_stabilizers = sum(1 for s in surface_eka.stabilizers if 'Z' in str(s))
    
    print(f"  X-type stabilizers: {x_stabilizers}")
    print(f"  Z-type stabilizers: {z_stabilizers}")
    
    print("\nThe surface code protects against both bit-flip (X) and phase-flip (Z) errors!")
    
except ImportError:
    print("Loom not installed.\n")
    print("Surface Code (Distance-3) Properties:")
    print("  Total qubits: 13 (9 data + 4 ancilla)")
    print("  X stabilizers: 2 (detect phase errors)")
    print("  Z stabilizers: 2 (detect bit-flip errors)")
    print("  Can correct any single-qubit error")

### Simulating the Surface Code with Stim

In [ ]:
try:
    from loom.backends.stim import StimBackend
    from loom.executor import Executor
    
    # Build a memory circuit
    surface_circuit = Circuit(surface_eka)
    surface_circuit.add_logical_initialization('Z')
    
    # Multiple QEC rounds
    for _ in range(5):
        surface_circuit.add_syndrome_measurement_round()
    
    surface_circuit.add_logical_measurement('Z')
    
    # Convert to Stim circuit
    stim_backend = StimBackend()
    stim_circuit = stim_backend.from_loom_circuit(surface_circuit)
    
    # Add noise
    error_rate = 0.001  # 0.1% error per operation
    noisy_circuit = stim_backend.add_noise(stim_circuit, error_rate=error_rate)
    
    # Run simulation
    executor = Executor(backend=stim_backend)
    results = executor.run(noisy_circuit, shots=1000)
    
    logical_error_rate = np.mean(results.logical_errors)
    
    print(f"\nSimulation Results:")
    print(f"  Physical error rate: {error_rate}")
    print(f"  Logical error rate: {logical_error_rate:.6f}")
    print(f"  Error suppression: {error_rate/logical_error_rate:.2f}x")
    print(f"  Shots: 1000")
    
except (ImportError, NameError):
    print("Stim simulation requires Loom + Stim installation")
    print("\nExpected behavior:")
    print("  Physical error rate: 0.001 (0.1%)")
    print("  Logical error rate: ~0.00001 (0.001%)")
    print("  Error suppression: ~100x")

<a id='loom-lattice'></a>
## 5. Loom: Lattice Surgery

Lattice surgery is a technique for performing logical operations on surface code patches by merging and splitting them. This is one of Loom's strongest features!

In [ ]:
try:
    from loom.lattice_surgery import LatticeSurgeryBlock
    
    print("Demonstrating lattice surgery: Logical CNOT gate\n")
    
    # Create two surface code patches
    patch1_factory = RotatedSurfaceCodeFactory(distance=3)
    patch2_factory = RotatedSurfaceCodeFactory(distance=3)
    
    patch1 = patch1_factory.generate_code()
    patch2 = patch2_factory.generate_code()
    
    # Create lattice surgery block
    ls_block = LatticeSurgeryBlock()
    
    # Position the patches
    ls_block.add_patch(patch1, position=(0, 0), label='control')
    ls_block.add_patch(patch2, position=(4, 0), label='target')
    
    print("Initial setup:")
    print("  Patch 1 (control) at position (0, 0)")
    print("  Patch 2 (target) at position (4, 0)")
    print("  Patches separated by 4 units\n")
    
    # Perform logical CNOT via lattice surgery
    # Steps: 1) Merge patches, 2) Measure, 3) Split patches
    ls_block.add_cnot_operation(control=patch1, target=patch2)
    
    # Convert to executable circuit
    ls_circuit = ls_block.to_circuit()
    
    print("Lattice Surgery CNOT:")
    print(f"  Circuit depth: {ls_circuit.depth} time steps")
    print(f"  Total operations: {len(ls_circuit.operations)}")
    print(f"  Merge duration: {ls_block.merge_time} steps")
    print(f"  Split duration: {ls_block.split_time} steps")
    
    print("\nThis demonstrates how Loom handles logical gates on surface codes!")
    
except ImportError:
    print("Loom lattice surgery module not available.\n")
    print("Lattice Surgery CNOT Concept:")
    print("  1. Merge: Physically connect two surface code patches")
    print("  2. Measure: Perform joint stabilizer measurements")
    print("  3. Split: Separate the patches back into individual logical qubits")
    print("  4. Result: Logical CNOT operation completed!")
    print("\nThis is MORE efficient than transversal operations for surface codes.")

### Visualizing Lattice Surgery with Entwine

Loom's **Entwine** GUI (https://entwine.entropicalabs.com/) provides a visual interface for designing lattice surgery circuits:

```
1. Drag surface code patches onto canvas
2. Click to connect patches for operations
3. Export the design as Loom Python code
4. Run simulations in this notebook!
```

This makes designing complex lattice surgery sequences much easier!

<a id='loom-eka'></a>
## 6. Loom: Custom Circuits with EKA

EKA (from Sanskrit "one") is Loom's core data structure providing fine-grained control over QEC experiments.

In [ ]:
try:
    from loom.eka import Eka, Qubit, Stabilizer
    from loom.block import Block
    
    print("Building a custom 3-qubit repetition code from scratch\n")
    
    # Create empty EKA
    custom_eka = Eka()
    
    # Define qubits
    q0 = Qubit(idx=0, type='data', position=(0, 0))
    q1 = Qubit(idx=1, type='data', position=(1, 0))
    q2 = Qubit(idx=2, type='data', position=(2, 0))
    a0 = Qubit(idx=3, type='ancilla', position=(0.5, -1))
    a1 = Qubit(idx=4, type='ancilla', position=(1.5, -1))
    
    # Add to EKA
    custom_eka.add_qubits([q0, q1, q2, a0, a1])
    
    print("Qubits defined:")
    print(f"  Data qubits: {[q.idx for q in [q0, q1, q2]]}")
    print(f"  Ancilla qubits: {[q.idx for q in [a0, a1]]}")
    
    # Define stabilizers
    # S1 = Z0⊗Z1 (checks if q0 and q1 have same value)
    # S2 = Z1⊗Z2 (checks if q1 and q2 have same value)
    s1 = Stabilizer(
        qubits=[q0, q1],
        pauli_string='ZZ',
        ancilla=a0,
        label='S1'
    )
    s2 = Stabilizer(
        qubits=[q1, q2],
        pauli_string='ZZ',
        ancilla=a1,
        label='S2'
    )
    
    custom_eka.add_stabilizers([s1, s2])
    
    print("\nStabilizers defined:")
    print(f"  {s1.label}: Z₀Z₁ (measured by ancilla {a0.idx})")
    print(f"  {s2.label}: Z₁Z₂ (measured by ancilla {a1.idx})")
    
    # Create operation blocks
    init_block = Block('initialization')
    init_block.add_operation('prepare_logical_zero', targets=[q0, q1, q2])
    
    syndrome_block = Block('syndrome_measurement')
    syndrome_block.add_syndrome_measurements([s1, s2])
    
    custom_eka.add_blocks([init_block, syndrome_block])
    
    print("\nEKA structure complete:")
    print(f"  Total qubits: {len(custom_eka.all_qubits)}")
    print(f"  Stabilizers: {len(custom_eka.stabilizers)}")
    print(f"  Operation blocks: {len(custom_eka.blocks)}")
    print("\nThis demonstrates the flexibility of Loom's EKA architecture!")
    
except ImportError:
    print("Loom EKA module not available.\n")
    print("EKA Concept:")
    print("  - Single source of truth for all QEC data")
    print("  - Stores qubits, stabilizers, operations")
    print("  - Fully serializable and queryable")
    print("  - Enables fine-grained control over QEC experiments")

---
# Part 3: Deltakit Examples

Deltakit by Riverlane provides a comprehensive SDK with focus on learning and deployment.

**Key Features:**
- Interactive textbook with 4 modules
- Cloud-connected proprietary decoders
- Advanced noise models (including leakage)
- qLDPC code support
- Deltaflow hardware integration

---

<a id='deltakit-setup'></a>
## 7. Deltakit: Setup & Repetition Code

First, let's set up Deltakit and create a simple repetition code.

In [ ]:
# Deltakit Setup
try:
    from deltakit import Client
    
    print("Setting up Deltakit...\n")
    print("NOTE: You need a free token from https://deltakit.riverlane.com/")
    print("Run this in your terminal first:")
    print('  python -c "from deltakit import Client; Client.set_token(\'YOUR_TOKEN\')")\n')
    
    # Initialize client
    client = Client()
    
    print(f"Deltakit client initialized")
    print(f"Cloud services available: {client.has_token}")
    
    if not client.has_token:
        print("\n⚠️  No token found. Some features will be limited.")
        print("Get your free token at: https://deltakit.riverlane.com/")
    
except ImportError:
    print("Deltakit not installed. Install with: pip install deltakit")
    print("Then get your free token at: https://deltakit.riverlane.com/")

In [ ]:
# Repetition Code with Deltakit
try:
    from deltakit import RepetitionCode
    
    print("Creating a distance-5 repetition code with Deltakit\n")
    
    # Create code
    dk_rep_code = RepetitionCode(distance=5)
    
    print(f"Code Properties:")
    print(f"  Distance: {dk_rep_code.distance}")
    print(f"  Physical qubits: {dk_rep_code.num_physical_qubits}")
    print(f"  Data qubits: {dk_rep_code.num_data_qubits}")
    print(f"  Ancilla qubits: {dk_rep_code.num_ancilla_qubits}")
    print(f"  Logical qubits encoded: {dk_rep_code.num_logical_qubits}")
    
    # Generate circuit
    circuit = dk_rep_code.generate_circuit()
    
    print(f"\nGenerated Circuit:")
    print(f"  Depth: {circuit.depth}")
    print(f"  Gates: {circuit.num_gates}")
    print(f"  Measurements: {circuit.num_measurements}")
    
except ImportError:
    print("Deltakit not available.\n")
    print("Distance-5 Repetition Code:")
    print("  Physical qubits: 9 (5 data + 4 ancilla)")
    print("  Can correct up to 2 errors")
    print("  Logical operations: Transversal X, measurement")

<a id='deltakit-memory'></a>
## 8. Deltakit: Memory Experiment

A memory experiment tests how well a code preserves quantum information over multiple QEC cycles.

In [ ]:
try:
    from deltakit import SurfaceCode, MemoryExperiment, NoiseModel
    
    print("Running a surface code memory experiment\n")
    
    # Create surface code
    dk_surface = SurfaceCode(distance=3)
    
    print(f"Surface Code:")
    print(f"  Distance: {dk_surface.distance}")
    print(f"  Physical qubits: {dk_surface.num_physical_qubits}")
    print(f"  X stabilizers: {dk_surface.num_x_stabilizers}")
    print(f"  Z stabilizers: {dk_surface.num_z_stabilizers}")
    
    # Setup experiment
    experiment = MemoryExperiment(
        code=dk_surface,
        num_rounds=10,
        initial_state='|0⟩'
    )
    
    # Define noise
    noise = NoiseModel(
        gate_error_rate=0.001,
        measurement_error_rate=0.001,
        idle_error_rate=0.0001
    )
    
    experiment.add_noise(noise)
    
    print(f"\nExperiment Setup:")
    print(f"  QEC rounds: 10")
    print(f"  Gate error rate: 0.1%")
    print(f"  Measurement error rate: 0.1%")
    print(f"  Initial state: |0⟩")
    
    # Run experiment
    print(f"\nRunning 1000 shots...")
    results = experiment.run(shots=1000)
    
    print(f"\nResults:")
    print(f"  Logical error rate: {results.logical_error_rate:.6f}")
    print(f"  Physical error rate: {noise.gate_error_rate}")
    print(f"  Error suppression: {noise.gate_error_rate/results.logical_error_rate:.2f}x")
    print(f"  Average syndrome weight: {results.avg_syndrome_weight:.2f}")
    
    # Plot results
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(results.logical_errors, bins=2, edgecolor='black')
    plt.xlabel('Logical Error (0=success, 1=error)')
    plt.ylabel('Count')
    plt.title('Logical Error Distribution')
    
    plt.subplot(1, 2, 2)
    rounds = range(1, len(results.error_rate_by_round) + 1)
    plt.plot(rounds, results.error_rate_by_round, marker='o')
    plt.xlabel('QEC Round')
    plt.ylabel('Logical Error Rate')
    plt.title('Error Rate Evolution')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Deltakit not available.\n")
    print("Memory Experiment Concept:")
    print("  1. Initialize logical qubit")
    print("  2. Run QEC cycles (measure syndromes, correct errors)")
    print("  3. Final measurement")
    print("  4. Compare to initial state")
    print("\nExpected: Logical errors << Physical errors")

<a id='deltakit-decoders'></a>
## 9. Deltakit: Decoder Comparison

Deltakit provides access to multiple state-of-the-art decoders. Let's compare them!

In [ ]:
try:
    from deltakit import Decoder, LCDDecoder, BPACDecoder
    import time
    
    print("Comparing different decoders\n")
    
    # Create a larger surface code for better comparison
    code = SurfaceCode(distance=5)
    
    # Setup experiment
    exp = MemoryExperiment(code=code, num_rounds=5)
    noise = NoiseModel(gate_error_rate=0.001)
    exp.add_noise(noise)
    
    # Test different decoders
    decoders = {
        'MWPM': Decoder('minimum_weight_perfect_matching'),
        'LCD': LCDDecoder(),  # Riverlane's proprietary decoder
        'BP-AC': BPACDecoder()  # Belief Propagation
    }
    
    print(f"Testing on distance-{code.distance} surface code")
    print(f"Physical error rate: {noise.gate_error_rate}\n")
    
    results_comparison = {}
    
    for name, decoder in decoders.items():
        print(f"Testing {name} decoder...")
        start_time = time.time()
        
        result = exp.run(shots=500, decoder=decoder)
        
        elapsed = time.time() - start_time
        
        results_comparison[name] = {
            'logical_error_rate': result.logical_error_rate,
            'decode_time_ms': result.avg_decode_time_ms,
            'total_time': elapsed
        }
        
        print(f"  Logical error rate: {result.logical_error_rate:.6f}")
        print(f"  Avg decode time: {result.avg_decode_time_ms:.3f} ms")
        print(f"  Total time: {elapsed:.2f} s\n")
    
    # Visualization
    plt.figure(figsize=(12, 4))
    
    # Error rates
    plt.subplot(1, 2, 1)
    names = list(results_comparison.keys())
    error_rates = [results_comparison[n]['logical_error_rate'] for n in names]
    plt.bar(names, error_rates, edgecolor='black')
    plt.ylabel('Logical Error Rate')
    plt.title('Decoder Accuracy Comparison')
    plt.yscale('log')
    
    # Decode times
    plt.subplot(1, 2, 2)
    decode_times = [results_comparison[n]['decode_time_ms'] for n in names]
    plt.bar(names, decode_times, edgecolor='black', color='orange')
    plt.ylabel('Decode Time (ms)')
    plt.title('Decoder Speed Comparison')
    
    plt.tight_layout()
    plt.show()
    
    print("Key Insights:")
    print("  - MWPM: Classical baseline, good accuracy")
    print("  - LCD: Riverlane's proprietary, optimized for real-time")
    print("  - BP-AC: Fast, scalable to qLDPC codes")
    
except ImportError:
    print("Deltakit not available.\n")
    print("Decoder Types:")
    print("  MWPM: Minimum Weight Perfect Matching")
    print("    - Classical algorithm")
    print("    - High accuracy")
    print("    - O(n³) complexity")
    print("\n  LCD: Local Clustering Decoder (Riverlane)")
    print("    - Proprietary")
    print("    - Optimized for real-time")
    print("    - Works with Deltaflow hardware")
    print("\n  BP-AC: Belief Propagation + Automorphism Clustering")
    print("    - Fast decoding")
    print("    - Scales well to qLDPC codes")
    print("    - Near-MWPM accuracy")

<a id='deltakit-qldpc'></a>
## 10. Deltakit: qLDPC Codes

Quantum Low-Density Parity-Check (qLDPC) codes are next-generation codes with better encoding rates.

In [ ]:
try:
    from deltakit import BivariateProductCode
    
    print("Creating a bivariate bicycle code (qLDPC)\n")
    
    # Create qLDPC code
    qldpc = BivariateProductCode(l=6, m=6)
    
    print(f"qLDPC Code Properties:")
    print(f"  Physical qubits: {qldpc.num_physical_qubits}")
    print(f"  Logical qubits: {qldpc.num_logical_qubits}")
    print(f"  Distance: {qldpc.distance}")
    print(f"  Encoding rate: {qldpc.encoding_rate:.3f}")
    print(f"  Check sparsity: {qldpc.check_sparsity:.3f}")
    
    # Compare to surface code
    surface_equiv = SurfaceCode(distance=qldpc.distance)
    
    print(f"\nComparison to Surface Code (same distance):")
    print(f"  Surface code physical qubits: {surface_equiv.num_physical_qubits}")
    print(f"  qLDPC physical qubits: {qldpc.num_physical_qubits}")
    print(f"  Qubit savings: {(1 - qldpc.num_physical_qubits/surface_equiv.num_physical_qubits)*100:.1f}%")
    
    # Generate circuit
    qldpc_circuit = qldpc.generate_circuit()
    
    print(f"\nCircuit Properties:")
    print(f"  Depth: {qldpc_circuit.depth}")
    print(f"  Two-qubit gates: {qldpc_circuit.num_two_qubit_gates}")
    print(f"  Connectivity: {qldpc_circuit.connectivity}")
    
    print("\nqLDPC codes are promising for future large-scale quantum computers!")
    print("They offer better qubit efficiency than surface codes.")
    
except ImportError:
    print("Deltakit qLDPC module not available.\n")
    print("qLDPC Code Benefits:")
    print("  ✅ Better encoding rate (more logical qubits per physical)")
    print("  ✅ Reduced qubit overhead")
    print("  ✅ Good distance scaling")
    print("\nChallenges:")
    print("  ⚠️  More complex decoding")
    print("  ⚠️  Non-local connectivity requirements")
    print("  ⚠️  Less mature than surface codes")

---
# Part 4: Comparative Analysis

Now let's compare Loom and Deltakit on various tasks!

---

<a id='comparison'></a>
## 11. Side-by-Side Comparison

Let's run the same QEC experiment using both frameworks and compare the results.

In [ ]:
# Comparison Task: Distance-3 surface code memory experiment

print("=" * 70)
print("SIDE-BY-SIDE COMPARISON: Loom vs Deltakit")
print("Task: Distance-3 surface code, 5 QEC rounds, 0.1% error rate")
print("=" * 70)

# === LOOM IMPLEMENTATION ===
print("\n--- LOOM Implementation ---")
try:
    from loom.code_factory import RotatedSurfaceCodeFactory
    from loom.circuit import Circuit
    from loom.backends.stim import StimBackend
    from loom.executor import Executor
    import time
    
    start = time.time()
    
    # Create code
    loom_factory = RotatedSurfaceCodeFactory(distance=3)
    loom_code = loom_factory.generate_code()
    loom_eka = loom_code.to_eka()
    
    # Build circuit
    loom_circuit = Circuit(loom_eka)
    loom_circuit.add_logical_initialization('Z')
    for _ in range(5):
        loom_circuit.add_syndrome_measurement_round()
    loom_circuit.add_logical_measurement('Z')
    
    # Simulate
    stim_backend = StimBackend()
    stim_circuit = stim_backend.from_loom_circuit(loom_circuit)
    stim_circuit = stim_backend.add_noise(stim_circuit, error_rate=0.001)
    
    executor = Executor(backend=stim_backend)
    loom_results = executor.run(stim_circuit, shots=1000)
    
    loom_time = time.time() - start
    loom_error_rate = np.mean(loom_results.logical_errors)
    
    print(f"✓ Setup time: {loom_time:.3f}s")
    print(f"✓ Logical error rate: {loom_error_rate:.6f}")
    print(f"✓ Circuit depth: {loom_circuit.depth}")
    print(f"✓ Backend: Stim")
    
except Exception as e:
    print(f"✗ Not available: {e}")
    loom_error_rate = None
    loom_time = None

# === DELTAKIT IMPLEMENTATION ===
print("\n--- DELTAKIT Implementation ---")
try:
    from deltakit import SurfaceCode, MemoryExperiment, NoiseModel
    import time
    
    start = time.time()
    
    # Create code
    dk_code = SurfaceCode(distance=3)
    
    # Setup experiment
    dk_exp = MemoryExperiment(code=dk_code, num_rounds=5)
    dk_noise = NoiseModel(gate_error_rate=0.001)
    dk_exp.add_noise(dk_noise)
    
    # Run
    dk_results = dk_exp.run(shots=1000)
    
    dk_time = time.time() - start
    dk_error_rate = dk_results.logical_error_rate
    
    print(f"✓ Setup time: {dk_time:.3f}s")
    print(f"✓ Logical error rate: {dk_error_rate:.6f}")
    print(f"✓ Decoder: {dk_results.decoder_used}")
    print(f"✓ Avg decode time: {dk_results.avg_decode_time_ms:.2f}ms")
    
except Exception as e:
    print(f"✗ Not available: {e}")
    dk_error_rate = None
    dk_time = None

# === COMPARISON ===
print("\n" + "=" * 70)
print("COMPARISON SUMMARY")
print("=" * 70)

if loom_error_rate and dk_error_rate:
    print(f"\nError Rate:")
    print(f"  Loom: {loom_error_rate:.6f}")
    print(f"  Deltakit: {dk_error_rate:.6f}")
    print(f"  Difference: {abs(loom_error_rate - dk_error_rate):.6f}")
    
    print(f"\nSetup Time:")
    print(f"  Loom: {loom_time:.3f}s")
    print(f"  Deltakit: {dk_time:.3f}s")
    
    print(f"\nKey Differences:")
    print("  Loom:")
    print("    + Fine-grained control via EKA")
    print("    + Visual design with Entwine")
    print("    + Multiple backends")
    print("  Deltakit:")
    print("    + Higher-level abstractions")
    print("    + Cloud decoders available")
    print("    + Built-in experiments")
else:
    print("Install both frameworks to run comparison!")

<a id='threshold'></a>
## 12. Threshold Analysis

The error threshold is a critical parameter in QEC. Below the threshold, increasing code distance reduces logical errors. Let's find it!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Threshold Analysis: How code distance affects logical error rates\n")

# Physical error rates to test
physical_error_rates = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03]
distances = [3, 5, 7]

# For demonstration, we'll use theoretical scaling
# In practice, you'd run actual simulations
print("Simulating threshold behavior (theoretical model)...\n")

results = {}
threshold_estimate = 0.01  # Typical surface code threshold ~1%

for d in distances:
    logical_errors = []
    for p in physical_error_rates:
        # Simplified model: logical error rate depends on distance and physical rate
        if p < threshold_estimate:
            # Below threshold: exponential improvement
            logical_p = p ** ((d + 1) / 2)
        else:
            # Above threshold: performance degrades
            logical_p = 1 - (1 - p) ** d
        logical_errors.append(logical_p)
    results[d] = logical_errors

# Plot
plt.figure(figsize=(12, 6))

for d in distances:
    plt.loglog(physical_error_rates, results[d], marker='o', label=f'Distance {d}', linewidth=2)

# Add threshold line
plt.axvline(threshold_estimate, color='red', linestyle='--', linewidth=2, label='Threshold (~1%)')

# Add reference line (no QEC)
plt.loglog(physical_error_rates, physical_error_rates, 'k:', linewidth=2, label='No QEC')

plt.xlabel('Physical Error Rate', fontsize=12)
plt.ylabel('Logical Error Rate', fontsize=12)
plt.title('QEC Threshold Behavior', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Key Observations:")
print("  1. Below threshold: Larger distance → Lower logical errors")
print("  2. Above threshold: QEC can make errors WORSE")
print(f"  3. Estimated threshold: ~{threshold_estimate*100:.1f}%")
print("  4. Surface codes have one of the highest known thresholds!")
print("\nThis is why improving physical qubit quality is crucial!")

<a id='benchmarks'></a>
## 13. Performance Benchmarks

Let's benchmark key operations in both frameworks.

In [ ]:
import time

print("=" * 70)
print("PERFORMANCE BENCHMARKS")
print("=" * 70)

benchmarks = {
    'Framework': [],
    'Code Creation': [],
    'Circuit Build': [],
    'Simulation (1000 shots)': [],
}

# Benchmark Loom
print("\nBenchmarking Loom...")
try:
    # Code creation
    start = time.time()
    factory = RotatedSurfaceCodeFactory(distance=5)
    code = factory.generate_code()
    eka = code.to_eka()
    code_time = time.time() - start
    
    # Circuit building
    start = time.time()
    circuit = Circuit(eka)
    circuit.add_logical_initialization('Z')
    for _ in range(3):
        circuit.add_syndrome_measurement_round()
    circuit.add_logical_measurement('Z')
    circuit_time = time.time() - start
    
    # Simulation
    start = time.time()
    backend = StimBackend()
    stim_circ = backend.from_loom_circuit(circuit)
    stim_circ = backend.add_noise(stim_circ, error_rate=0.001)
    executor = Executor(backend=backend)
    results = executor.run(stim_circ, shots=1000)
    sim_time = time.time() - start
    
    benchmarks['Framework'].append('Loom')
    benchmarks['Code Creation'].append(f"{code_time*1000:.2f}ms")
    benchmarks['Circuit Build'].append(f"{circuit_time*1000:.2f}ms")
    benchmarks['Simulation (1000 shots)'].append(f"{sim_time:.3f}s")
    
    print(f"✓ Code creation: {code_time*1000:.2f}ms")
    print(f"✓ Circuit build: {circuit_time*1000:.2f}ms")
    print(f"✓ Simulation: {sim_time:.3f}s")
    
except Exception as e:
    print(f"✗ Error: {e}")

# Benchmark Deltakit
print("\nBenchmarking Deltakit...")
try:
    # Code creation
    start = time.time()
    dk_code = SurfaceCode(distance=5)
    code_time = time.time() - start
    
    # Experiment setup (equivalent to circuit building)
    start = time.time()
    exp = MemoryExperiment(code=dk_code, num_rounds=3)
    noise = NoiseModel(gate_error_rate=0.001)
    exp.add_noise(noise)
    circuit_time = time.time() - start
    
    # Simulation
    start = time.time()
    results = exp.run(shots=1000)
    sim_time = time.time() - start
    
    benchmarks['Framework'].append('Deltakit')
    benchmarks['Code Creation'].append(f"{code_time*1000:.2f}ms")
    benchmarks['Circuit Build'].append(f"{circuit_time*1000:.2f}ms")
    benchmarks['Simulation (1000 shots)'].append(f"{sim_time:.3f}s")
    
    print(f"✓ Code creation: {code_time*1000:.2f}ms")
    print(f"✓ Experiment setup: {circuit_time*1000:.2f}ms")
    print(f"✓ Simulation: {sim_time:.3f}s")
    
except Exception as e:
    print(f"✗ Error: {e}")

# Display results table
if benchmarks['Framework']:
    print("\n" + "=" * 70)
    print("BENCHMARK RESULTS (Distance-5 Surface Code, 3 rounds)")
    print("=" * 70)
    
    import pandas as pd
    df = pd.DataFrame(benchmarks)
    print(df.to_string(index=False))
    
    print("\nNotes:")
    print("  - Times may vary based on system and installation")
    print("  - Both frameworks are highly optimized")
    print("  - Loom: Benefits from Stim's fast stabilizer simulation")
    print("  - Deltakit: Can use cloud decoders for better performance")
else:
    print("\nInstall frameworks to run benchmarks!")

---
## Summary and Recommendations

### When to Use Loom:
✅ **Lattice Surgery**: Extensive support for surface code operations  
✅ **Visual Design**: Entwine GUI for drag-and-drop circuit creation  
✅ **Research**: Fine-grained control via EKA architecture  
✅ **Custom Codes**: Build novel QEC codes from scratch  
✅ **Backend Flexibility**: Support for Stim, Qiskit, OpenQASM  

### When to Use Deltakit:
✅ **Learning**: Interactive textbook with guided exercises  
✅ **Production**: Built-in experiments and workflows  
✅ **Advanced Decoders**: Access to LCD and BP-AC  
✅ **qLDPC Codes**: Better support for next-gen codes  
✅ **Hardware Deployment**: Seamless Deltaflow integration  
✅ **Leakage**: Sophisticated leakage modeling  

### Best Practice:
Use **both** frameworks! Many practitioners:
1. Learn fundamentals with **Deltakit's textbook**
2. Prototype visually with **Loom's Entwine**
3. Optimize with **Deltakit's decoders**
4. Deploy via **Deltaflow** or custom backends

---

## Resources

### Loom (Entropica Labs)
- Documentation: https://loom-api-docs.entropicalabs.com/
- Design Guide: https://loom-docs.entropicalabs.com/
- Entwine GUI: https://entwine.entropicalabs.com/
- GitHub: https://github.com/entropicalabs/el-loom
- Contact: info@entropicalabs.com

### Deltakit (Riverlane)
- Website: https://deltakit.riverlane.com/
- Interactive Textbook: https://deltakit.riverlane.com/textbook
- Documentation: https://deltakit.riverlane.com/docs
- Get Free Token: https://deltakit.riverlane.com/
- Deltaflow Info: https://www.riverlane.com/deltaflow

### General QEC Resources
- Qiskit Textbook QEC Chapter
- Nielsen & Chuang: "Quantum Computation and Quantum Information"
- Fowler et al.: "Surface codes: Towards practical large-scale quantum computation"
- Gottesman: "Stabilizer Codes and Quantum Error Correction" (PhD thesis)

---

## Next Steps

1. **Install the frameworks:**
   ```bash
   pip install loom deltakit
   ```

2. **Get Deltakit token:**
   - Visit https://deltakit.riverlane.com/
   - Create free account
   - Generate token

3. **Try Entwine:**
   - Go to https://entwine.entropicalabs.com/
   - Design a lattice surgery circuit
   - Export to Loom code

4. **Complete Deltakit textbook:**
   - Module 1: Introduction to QEC
   - Module 2: Repetition codes
   - Module 3: Surface codes
   - Module 4: Decoding

5. **Experiment:**
   - Modify the examples in this notebook
   - Try different code distances
   - Compare decoders
   - Build custom codes

6. **Join the community:**
   - Loom: GitHub discussions
   - Deltakit: Community forums
   - QEC conferences and workshops

**Happy quantum error correcting!** 🎉🔬✨